## Kelvin Representation of the Standard Linear Solid. $G_0$ and $G_1$ are the spring stiffness, $\eta_1$ the dashpot damping coefficient.

![](figures/Representation%20of%20the%20Standard%20Linear%20Solid.jpg)

### Prony series

In a one-dimensional relaxation test, the material is subjected to a sudden strain that is kept constant over the duration of the test, and the stress is measured over time. The initial stress is due to the elastic response of the material. Then, the stress relaxes over time due to the viscous effects in the material. Typically, either a tensile, compressive, bulk compression, or shear strain is applied. The resulting stress vs. time data can be fitted with a number of equations, called models. Only the notation changes depending on the type of strain applied: tensile-compressive relaxation is denoted $E$, shear is denoted $G$, bulk is denoted $K$. The Prony series for the shear relaxation is[3]:

$$
G(t)=G_{\infty}+\sum_{i=1}^N G_i \exp \left(-t / \tau_i\right)
$$
where $G_{\infty}$ is the long term modulus once the material is totally relaxed, $\tau_i$ are the relaxation times (not to be confused with $\tau_i$ in the diagram); the higher their values, the longer it takes for the stress to relax.

#### We first test our model for one brunch Kervin model($k=1$):

The relaxation time $\tau_1$ (or called the characteristic time of the creep compliance function[2]) is typically defined as the ratio of the dashpot's viscosity $\eta_1$ to the spring's modulus $G_1$ that it is in parallel with, not the series spring modulus $G_0$. So, it would be:

$$
\tau_1 = \frac{\eta_1}{G_1}
$$

While defining the parameters, we can choose to give $\left(G_{\infty}, G_i, \tau_i\right)$ or $\left(G_{\infty}, G_i, \eta_i\right)$

To clarify, $k=1$ means that we use one-branch model to simulate the contact evolution between two viscoelastic rough surface with similar timescale $\tau$. And definitely, we simplify this model into the contact between one flat viscoelastic surface and one equivalent rigid rough surface.

### Viscoelastic model for one branch($k=1$)

An incompressible viscoelastic material with a Zener law follows the differential equation:


$$
s\left(1+\frac{G_1}{G_0}\right)+\frac{\eta_1}{G_0} \dot{s}=2 G_1 e+2 \eta_1 \dot{e}
$$
where $s$ and $e$ are the deviatoric parts of the stress and strain tensors, the dot denotes a time derivative.[1]

The critical part of our implementation for contact step in our algorithm relates to the Laplace transformation. In the Laplace domain, the problem is equivalent to an elastic contact problem, which means it is possible to use analytical elastic solutions, in particular the Boussinesq potential.[1]

After the Laplace transformation(formula (5) in [1]) and a BEM discretization(formula (6) in [1]), we can discretize the integration formula(formula (6) in [1]) and end up with:
$$
A_{z z} P_{t+\delta t}\left(1+\frac{G_1}{G_0}\right)+\frac{\eta_1}{G_0} A_{z z} \frac{\Delta P}{\Delta t}=G_1 U_{t+\delta t}+\eta_1 \frac{\Delta U}{\Delta t}
$$

It is then possible to use the 'elastic' Conjugate Gradient contact solver using $P^{\prime}, U_z^{\prime}$ and $H^{\prime}$ instead of $P, U_z$ and $H$ according to:

$$
P^{\prime}=\sum_k P_{t+\Delta t}^k
$$

$$
U_z^{\prime}=A_{z z} P^{\prime}
$$

$$
\begin{aligned}
H^{\prime}= & H\left[G_{\infty}+\sum_k \frac{G_1^k+\frac{\eta_1^k}{\Delta t}}{1+\frac{G_1^k}{G_0^k}+\frac{\eta_1^k}{G_0^k \Delta t}}\right]-U_t \sum_k \frac{\frac{\eta_1^k}{\Delta t}}{1+\frac{G_1^k}{G_0^k}+\frac{\eta_1^k}{G_0^k \Delta t}} \\
& +\sum_k A_{z z} P_t^k \frac{\frac{\eta_1^k}{G_0^k \Delta t}}{1+\frac{G_1^k}{G_0^k}+\frac{\eta_1^k}{G_0^k \Delta t}}
\end{aligned}
$$

This yields $U_{t+\Delta t}$ and $\sum_k P_{t+\Delta t}^k$, which is the total pressure acting on the surface. Here we just implement **Steepest decent** algorithm, and we will update in the next version of our solver.

## Viscoelastic contact routine (Bugnicourt et al. 2017)

- **Input:** 
  - External load $W_{t+\Delta t}$
  - Timestep $\Delta t$
  - Partial displacements $\mathbb{M}_t^k = A_{z z} P_t^k$
  - Displacement $\mathbb{U}_t$
  - Surface $\mathbb{H}$
  - Material properties $G_\infty$, $G_{0,1}^k$, $\eta^k$

- **Steps:**
  1. Calculate $\alpha$ as follows:
     $$\alpha = G_\infty + \sum_k{\frac{G_1^k + \frac{\eta^k}{\Delta t}}{1 + \frac{G_1^k}{G_0^k} + \frac{\eta^k}{G_0^k\Delta t}}}$$
  2. Calculate $\beta$ as follows:
     $$\beta = \sum_k{\frac{\frac{\eta^k}{\Delta t}}{1 + \frac{G_1^k}{G_0^k} + \frac{\eta^k}{G_0^k\Delta t}}}$$
  3. For each $k$, calculate $\gamma^k$ as follows:
     $$\gamma^k = \frac{\frac{\eta^k}{G_0^k\Delta t}}{1 + \frac{G_1^k}{G_0^k} + \frac{\eta^k}{G_0^k\Delta t}}$$
  4. Update $\mathbb{H}_{t+\Delta t}$ using $\alpha$, $\beta$, and $\gamma^k$:
     $$\mathbb{H}_{t+\Delta t} = \alpha\mathbb{H} + \beta \mathbb{U}_t + \sum_k \gamma^k \mathbb{M}_t^k$$
  5. Solve elastic contact for $W_{t + \Delta t}$ and $\mathbb{H}_{t + \Delta t}$, obtain $\mathbb{U}_{t + \Delta t}$
  6. Solve for $\mathbb{M}_{t + \Delta t}^k$:
     $$\mathbb{M}_{t + \Delta t}^k \left(1 + \frac{G_1^k}{G_0^k}\right) + \frac{\eta^k}{G_0^k\Delta t}(\mathbb{M}_{t + \Delta t}^k - \mathbb{M}_t^k) = G_1^k \mathbb{U}_{t + \Delta t} + \frac{\eta^k}{\Delta t}(\mathbb{U}_{t + \Delta t} - \mathbb{U}_t)$$



#### Just to summarize, in this backward Euler procedure(for loop), we mainly have three updating steps:

##### Step0: Update the effective modulus

In our existed solver, we need effective modulus, here we apply Prony series with $k=1$:

$$
G(t)=G_{\infty}+G_1 \exp \left(-t / \tau_0\right)
$$

##### Step1: Update surface profile $H_t$
##### Step2: Update displacement $U_t$ with our existed solver
##### Step3: Update partial displacement $M^k_t$ as integration of $P^k_t$(no need to update $P^k_t$ directly)

### Parameter setting

### Test for $k = 1$




## Reference:

[1] Bugnicourt, R., P. Sainsot, N. Lesaffre, and A.A. Lubrecht. ‘Transient Frictionless Contact of a Rough Rigid Surface on a Viscoelastic Half-Space’. Tribology International 113 (September 2017): 279–85. https://doi.org/10.1016/j.triboint.2017.01.032.

[2] Marques, Severino P. C., and Guillermo J. Creus. Computational Viscoelasticity. SpringerBriefs in Applied Sciences and Technology. Berlin, Heidelberg: Springer Berlin Heidelberg, 2012. https://doi.org/10.1007/978-3-642-25311-9.

[3] https://en.wikipedia.org/wiki/Viscoelasticity